In [ ]:
#!pip install openai
#!pip install python-dotenv

In [3]:
from openai import OpenAI
import os

#client.api_key = os.getenv('OPENAI_API_KEY') # asigna la API key de chatGPT

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # lee el archivo .env donde está la api key


client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY")
)

In [4]:
def get_completion(prompt):
    completion = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content

In [5]:
get_completion("como te llamas?")

'Soy una inteligencia artificial y no tengo un nombre propio. Estoy aquí para ayudarte, ¿en qué puedo asistirte?'

In [6]:
#!pip install --upgrade langchain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [8]:
llm = ChatOpenAI(temperature=0.0)

In [9]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [10]:
conversation.predict(input="cual es mi nombre?")

'Lo siento, pero no tengo acceso a tu información personal, por lo que no puedo saber cuál es tu nombre.'

In [11]:
print(memory.buffer)

Human: cual es mi nombre?
AI: Lo siento, pero no tengo acceso a tu información personal, por lo que no puedo saber cuál es tu nombre.


In [12]:
from pathlib import Path
import openai
from IPython.display import Audio

# Define the path for the output file
speech_file_path = Path("speech.mp3")

# Call the OpenAI API to generate speech
response = openai.audio.speech.create(
  model="tts-1",
  voice="echo",
  input="hola mi nombre es Mario"
)

# Stream the response to the specified file
response.stream_to_file(speech_file_path)

# Play the audio in the notebook
Audio(str(speech_file_path))

In [13]:


user_input = input("Query: ")

respuesta = conversation.predict(input=user_input)

# Define the path for the output file
speech_file_path = Path("speech.mp3")

# Call the OpenAI API to generate speech
response = openai.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=respuesta
)

# Stream the response to the specified file
response.stream_to_file(speech_file_path)

# Play the audio in the notebook
Audio(str(speech_file_path))

Query: sdf


In [14]:
#!pip install git+https://github.com/openai/whisper.git

In [15]:
#!pip show sounddevice

In [16]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import queue

fs = 44100  # Sample rate
chunk_size = 1024  # Size of each audio chunk
silence_threshold = 0.02  # Adjust this based on your noise conditions
silence_duration = 2  # Duration of silence in seconds to stop recording
recording_timeout = 10  # Maximum recording duration in seconds

duration = 0.2  # seconds
frequency = 440  # Hz
t = np.linspace(0, duration, int(fs * duration), endpoint=False)
beep_sound = np.sin(2 * np.pi * frequency * t)

# Load start and end sounds
start_sound, _ = wav.read('start_sound.wav')
end_sound, _ = wav.read('end_sound.wav')

# Function to play a sound
def play_sound(sound):
    sd.play(sound, fs)
    sd.wait()

# Queue to hold audio chunks
audio_queue = queue.Queue()

# Callback function to capture audio chunks
def callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

# Open a stream for continuous audio input
stream = sd.InputStream(callback=callback, channels=2, samplerate=fs, blocksize=chunk_size)
stream.start()

# Variables for managing recording state
recording = False
data = []
silence_counter = 0

print("Monitoring for voice activity...")


try:
    while True:
        chunk = audio_queue.get()
        if np.max(np.abs(chunk)) > silence_threshold:
            if not recording:
                print("Starting recording...")
                 # Play start sound
                play_sound(beep_sound)
                recording = True
                data = [chunk]
            else:
                data.append(chunk)
            silence_counter = 0
        elif recording:
            silence_counter += chunk_size / fs
            data.append(chunk)
            if silence_counter >= silence_duration:
                print("Stopping recording due to silence.")
                play_sound(beep_sound)  # Play end sound
                break
except KeyboardInterrupt:
    print("\nRecording stopped manually.")

stream.stop()
stream.close()

# Combine chunks into a single array and save as WAV file
if data:
    recording_data = np.concatenate(data, axis=0)
    wav.write('output.wav', fs, recording_data)
    print("Recording saved as 'output.wav'")
else:
    print("No voice activity detected.")

Monitoring for voice activity...
Starting recording...
Stopping recording due to silence.
Recording saved as 'output.wav'


In [17]:
import whisper

model = whisper.load_model("base")  # or another model size based on your preference
result = model.transcribe("output.wav")
print(result["text"])

C:\Users\Mario\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 como estás.


In [23]:
import scipy.io.wavfile as wav
import numpy as np
import whisper


fs = 44100  # Sample rate
chunk_size = 1024  # Size of each audio chunk
silence_threshold = 0.02  # Adjust this based on your noise conditions
silence_duration = 2  # Duration of silence in seconds to stop recording
recording_timeout = 30  # Maximum recording duration in seconds

duration = 0.2  # seconds
frequency = 440  # Hz
t = np.linspace(0, duration, int(fs * duration), endpoint=False)
beep_sound = np.sin(2 * np.pi * frequency * t)

# Function to play a sound
def play_sound(sound):
    sd.play(sound, fs)
    sd.wait()
    
def get_audio():
    
    # Queue to hold audio chunks
    audio_queue = queue.Queue()

    # Callback function to capture audio chunks
    def callback(indata, frames, time, status):
        audio_queue.put(indata.copy())

    # Open a stream for continuous audio input
    stream = sd.InputStream(callback=callback, channels=2, samplerate=fs, blocksize=chunk_size)
    stream.start()

    # Variables for managing recording state
    recording = False
    data = []
    silence_counter = 0

    print("Monitoring for voice activity...")


    try:
        while True:
            chunk = audio_queue.get()
            if np.max(np.abs(chunk)) > silence_threshold:
                if not recording:
                    print("Starting recording...")
                     # Play start sound
                    play_sound(beep_sound)
                    recording = True
                    data = [chunk]
                else:
                    data.append(chunk)
                silence_counter = 0
            elif recording:
                silence_counter += chunk_size / fs
                data.append(chunk)
                if silence_counter >= silence_duration:
                    print("Stopping recording due to silence.")
                    play_sound(beep_sound)  # Play end sound
                    break
    except KeyboardInterrupt:
        print("\nRecording stopped manually.")

    stream.stop()
    stream.close()

    # Combine chunks into a single array and save as WAV file
    if data:
        recording_data = np.concatenate(data, axis=0)
        wav.write('output.wav', fs, recording_data)
        print("Recording saved as 'output.wav'")
        model = whisper.load_model("base")  # or another model size based on your preference
        result = model.transcribe("output.wav")
        print(result["text"])
    else:
        print("No voice activity detected.")
        result = "lo siento, no logré escuchar"
    
    return result["text"]

    

In [24]:
from IPython.display import Audio, display
from pathlib import Path
import time

while True:
    #user_input = input("Query: ")
    user_input = get_audio()

    if user_input == "salir":
        break

    respuesta = conversation.predict(input=user_input)
    
    palabras = len(respuesta.split())
    
    espera = (palabras*60) / 150
    print(espera+2)

    # Define the path for the output file
    speech_file_path = Path("speech.mp3")

    # Call the OpenAI API to generate speech
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=respuesta
    )

    # Stream the response to the specified file
    response.stream_to_file(speech_file_path)

    # Play the audio in the notebook automatically
    audio = Audio(str(speech_file_path), autoplay=True)
    display(audio)
    time.sleep(espera)
    



Monitoring for voice activity...
Starting recording...
Stopping recording due to silence.
Recording saved as 'output.wav'


C:\Users\Mario\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 robots, namely robots that work closely together with people, sharing a workplace without actually producing harm to people. And we really focus on the promotion of a good mental health of workers. This is a European project together with many partners from Italy and KUKA, one of the biggest robotics companies in the world, is also involved as a partner. And just to show you that actually the spectrum is extremely broad, so I guess there.
37.6


Monitoring for voice activity...
Starting recording...
Stopping recording due to silence.
Recording saved as 'output.wav'


C:\Users\Mario\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Monitoring for voice activity...
Starting recording...
Stopping recording due to silence.
Recording saved as 'output.wav'


C:\Users\Mario\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hola, está pensando que tal vez sería buena idea que hablaras español, que opinas de esto.
16.8


Monitoring for voice activity...

Recording stopped manually.
No voice activity detected.


TypeError: string indices must be integers, not 'str'

## EXTRA CODE

In [ ]:
def get_audio():
    
    # Open a stream for continuous audio input
    stream = sd.InputStream(callback=callback, channels=2, samplerate=fs, blocksize=chunk_size)
    stream.start()

    # Variables for managing recording state
    recording = False
    data = []
    silence_counter = 0
    
    print("Monitoring for voice activity...")

    try:
        while True:
            chunk = audio_queue.get()
            if np.max(np.abs(chunk)) > silence_threshold:
                if not recording:
                    print("Starting recording...")
                     # Play start sound
                    play_sound(beep_sound)
                    recording = True
                    data = [chunk]
                else:
                    data.append(chunk)
                silence_counter = 0
            elif recording:
                silence_counter += chunk_size / fs
                data.append(chunk)
                if silence_counter >= silence_duration:
                    print("Stopping recording due to silence.")
                    play_sound(beep_sound)  # Play end sound
                    break
    except KeyboardInterrupt:
        print("\nRecording stopped manually.")

    stream.stop()
    stream.close()

    # Combine chunks into a single array and save as WAV file
    if data:
        recording_data = np.concatenate(data, axis=0)
        wav.write('output.wav', fs, recording_data)
        print("Recording saved as 'output.wav'")
    else:
        print("No voice activity detected.")
        
        
    model = whisper.load_model("base")  # or another model size based on your preference
    result = model.transcribe("output.wav")
    print(result["text"])
    return result["text"]

In [ ]:
from IPython.display import Audio, display
from pathlib import Path
import threading
import sounddevice as sd
import numpy as np
import wavio
import whisper
import queue

# Define global variables
fs = 44100  # Sample rate
chunk_size = 1024  # Chunk size for audio input
silence_threshold = 0.01  # Adjust this threshold as needed
silence_duration = 1.0  # Adjust this duration as needed
audio_queue = queue.Queue()

def callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    audio_queue.put(np.copy(indata))


while True:
    user_input = get_audio()

    if user_input == "salir":
        break

    respuesta = conversation.predict(input=user_input)

    # Define a function for generating audio in a separate thread
    def generate_audio():
        response = openai.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=respuesta
        )
        speech_file_path = Path("speech.mp3")
        response.stream_to_file(speech_file_path)

        # Play the audio in the notebook automatically
        audio = Audio(str(speech_file_path), autoplay=True)
        display(audio)

    # Create a thread to generate audio
    audio_thread = threading.Thread(target=generate_audio)
    audio_thread.start()

    audio_thread.join()  # Wait for the audio generation thread to finish

# ... (The rest of your code)


In [ ]:
!pip install wavio


In [ ]:
from IPython.display import Audio, display
from pathlib import Path
import threading
import sounddevice as sd
import numpy as np
import wavio
import whisper
import queue

# Define global variables
fs = 44100  # Sample rate
chunk_size = 1024  # Chunk size for audio input
silence_threshold = 0.01  # Adjust this threshold as needed
silence_duration = 1.0  # Adjust this duration as needed
audio_queue = queue.Queue()

def callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    audio_queue.put(np.copy(indata))

# Event to signal when audio playback is complete
audio_playback_complete = threading.Event()

while True:
    user_input = get_audio()

    if user_input == "salir":
        break

    respuesta = conversation.predict(input=user_input)

    # Define a function for generating audio in a separate thread
    def generate_audio():
        response = openai.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=respuesta
        )
        speech_file_path = Path("speech.mp3")
        response.stream_to_file(speech_file_path)

        # Play the audio in the notebook automatically
        audio = Audio(str(speech_file_path), autoplay=True)
        display(audio)

        # Signal that audio playback is complete
        audio_playback_complete.set()

    # Create a thread to generate audio
    audio_thread = threading.Thread(target=generate_audio)
    audio_thread.start()

    # Wait for the audio playback event to be set
    audio_playback_complete.wait()

# ... (The rest of your code)


In [ ]:
!pip install pydub


In [ ]:
!pip install pygame
